<a href="https://colab.research.google.com/github/martinpius/Applied-Predictive-Modeling2/blob/master/Univariate_time_series_Modeling_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
try:
  drive.mount("/content/drive/", force_remount = True)
  COLAB = True
  import tensorflow
  print(f"You are using google colab with tensorflow version: {tensorflow.__version__}")
except:
  COLAB = False
  print("You are not using Colab!!!")
  

Mounted at /content/drive/
You are using google colab with tensorflow version: 2.3.0


In [2]:
def time_setter(x):
  hours = int(x/(60 * 60))
  minutes = int(x%(60*60)/60)
  seconds = int(x%60)
  return f"{hours}: {minutes:>02}: {seconds:>05.2f}"

In [6]:
#A function to creating time stamps for univariate time series data.
def time_stamp_gen(data, time_stamps=1):
  x, y = [], []
  for i in range(len(data)):
    index = i + time_stamps
    if index > len(data) - 1:
      break
    x1, y1 = data[i:index], data[index]
    x.append(x1)
    y.append(y1)
  return np.array(x), np.array(y)

In [7]:
#Testing the function
stock = [123,124,122,178,223,327,234,345,343,333,433]
x, y = time_stamp_gen(stock, 3)

In [8]:
(x, y)

(array([[123, 124, 122],
        [124, 122, 178],
        [122, 178, 223],
        [178, 223, 327],
        [223, 327, 234],
        [327, 234, 345],
        [234, 345, 343],
        [345, 343, 333]]), array([178, 223, 327, 234, 345, 343, 333, 433]))

In [9]:
#Importing the necessary packages
import time, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization, LSTM, Bidirectional,Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from sklearn import metrics
from sklearn.model_selection import train_test_split

plt.style.use('fivethirtyeight')
%matplotlib inline


In [11]:
#Create fake data
stock = np.empty(10000)
for i in range(10000):
  stock[i] = np.random.random()
    

In [12]:
stock  = list(stock)

In [13]:
#Get the time stamps from our data
x, y = time_stamp_gen(stock,3)

In [14]:
x[0:10]

array([[0.46330715, 0.49381954, 0.84259213],
       [0.49381954, 0.84259213, 0.32133481],
       [0.84259213, 0.32133481, 0.67212836],
       [0.32133481, 0.67212836, 0.82126932],
       [0.67212836, 0.82126932, 0.09141631],
       [0.82126932, 0.09141631, 0.38799846],
       [0.09141631, 0.38799846, 0.26093111],
       [0.38799846, 0.26093111, 0.30749462],
       [0.26093111, 0.30749462, 0.70747326],
       [0.30749462, 0.70747326, 0.12794051]])

In [15]:
y[0:10]

array([0.32133481, 0.67212836, 0.82126932, 0.09141631, 0.38799846,
       0.26093111, 0.30749462, 0.70747326, 0.12794051, 0.14301224])

In [16]:
#Reshape the data into appropriate input for rnn
x = x.reshape(x.shape[0],x.shape[1],1)


In [17]:
x.shape

(9997, 3, 1)

In [18]:
y.shape

(9997,)

In [19]:
#Splitting the data into train-validation
x_train,x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 123)

In [20]:
print(f"train_shape: {x_train.shape}\n y_train_shape: {y_train.shape}\nx_test_shape: {x_test.shape}\ntest_y_shape: {y_test.shape}")

train_shape: (7997, 3, 1)
 y_train_shape: (7997,)
x_test_shape: (2000, 3, 1)
test_y_shape: (2000,)


In [22]:
#Building the model
model = Sequential()
model.add(Input(shape = (3,1)))
model.add(Bidirectional(LSTM(units = 256, dropout=0.2,activation = 'relu',recurrent_dropout= 0.2,return_sequences= True)))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(LSTM(units =128, dropout = 0.2, activation = 'relu',recurrent_dropout= 0.2, return_sequences= True))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(loss ='mean_squared_error', optimizer='RMSprop')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 3, 512)            528384    
_________________________________________________________________
batch_normalization (BatchNo (None, 3, 512)            2048      
_________________________________________________________________
dropout (Dropout)            (None, 3, 512)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 3, 128)            328192    
_________________________________________________________________
batch_normalization_1 (Batch (None, 3, 128)            512       
_________________________________________________________________
dropout_1 (Dropout)          (None, 3, 128)            0         
_________________________________________________________________
dense (Dense)                (None, 3, 1)              1

In [23]:
info = EarlyStopping(monitor = 'val_loss', min_delta = 1e-3, verbose = 1, patience = 100,restore_best_weights=True, mode = 'auto')

In [24]:
model.fit(x_train, y_train,validation_data=(x_test,y_test), epochs = 1000, verbose = 2, callbacks=[info], batch_size= 64)

Epoch 1/1000
125/125 - 5s - loss: 0.4991 - val_loss: 0.0851
Epoch 2/1000
125/125 - 5s - loss: 0.0855 - val_loss: 0.0848
Epoch 3/1000
125/125 - 5s - loss: 0.0837 - val_loss: 0.0846
Epoch 4/1000
125/125 - 4s - loss: 0.0836 - val_loss: 0.0849
Epoch 5/1000
125/125 - 4s - loss: 0.0836 - val_loss: 0.0846
Epoch 6/1000
125/125 - 5s - loss: 0.0834 - val_loss: 0.0846
Epoch 7/1000
125/125 - 4s - loss: 0.0834 - val_loss: 0.0851
Epoch 8/1000
125/125 - 5s - loss: 0.0834 - val_loss: 0.0847
Epoch 9/1000
125/125 - 4s - loss: 0.0832 - val_loss: 0.0846
Epoch 10/1000
125/125 - 4s - loss: 0.0833 - val_loss: 0.0846
Epoch 11/1000
125/125 - 5s - loss: 0.0833 - val_loss: 0.0857
Epoch 12/1000
125/125 - 4s - loss: 0.0834 - val_loss: 0.0847
Epoch 13/1000
125/125 - 4s - loss: 0.0832 - val_loss: 0.0851
Epoch 14/1000
125/125 - 4s - loss: 0.0833 - val_loss: 0.0846
Epoch 15/1000
125/125 - 4s - loss: 0.0832 - val_loss: 0.0849
Epoch 16/1000
125/125 - 4s - loss: 0.0833 - val_loss: 0.0846
Epoch 17/1000
125/125 - 4s - loss